In [ ]:
import numpy as np
import matplotlib
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.pyplot import subplots
from scipy.signal import convolve
from mpl_toolkits.mplot3d import Axes3D
from numpy.fft import fftn, fftshift
import glob
from tqdm.auto import tqdm
from multiprocess import Pool
import os
# import plotly.graph_objects as go
from PIL import Image
%matplotlib widget

In [ ]:
from tools.ptable_dict import ptable, atomic_masses, inverse_ptable, aff_dict
from tools.utilities import write_xyz, load_xyz, rotation_matrix, gaussian_kernel
from tools.voxelgrids import generate_density_grid, convert_grid_qspace, plot_3D_grid, downselect_meshgrid, multiply_ft_gaussian
from tools.detector import make_detector, rotate_about_normal, rotate_about_horizontal, rotate_about_vertical
from tools.detector import intersect_detector, rotate_psi_phi_theta, mirror_vertical_horizontal, generate_detector_ints
# from tools.giwaxs_comparison import mask_forbidden_pixels, mirror_qmap_positive_qxy_only, normalize_qmap, rebin_and_combine_qmaps
# from tools.giwaxs_comparison import add_f0_q_dependence

In [ ]:
def iq_to_mp4(psis, phis, thetas, iq, qx, qy, qz, output_path, frame_rate=15, quality=17):
    """
    Generate mp4 video of images along a specified dimension (e.g. energy, time). 
    Requires subprocess import. 
    
    Inputs:
    output_path (str or pathlib.Path): path to generated mp4 (includes mp4 filename)
    frame_rate (int, default=15): frame rate of mp4 generated
    quality (int, default=17): 'crf' quality value; lower is better, 17 is often considered visually lossless
    
    Outputs:
    mp4 movie file where specified in output path
    """

    # FFmpeg command. This is set up to accept data from the pipe and use it as input, with PNG format.
    # It will then output an H.264 encoded MP4 video.
    cmd = [
        'ffmpeg',
        '-y',  # Overwrite output file if it exists
        '-f', 'image2pipe',
        '-vcodec', 'png',
        '-r', str(frame_rate),  # Frame rate
        '-i', '-',  # The input comes from a pipe
        '-vcodec', 'libx264',
        '-pix_fmt', 'yuv420p',
        '-crf', str(quality),  # Set the quality (lower is better, 17 is often considered visually lossless)
        str(output_path)
    ]

    # Start the subprocess
    proc = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Loop through the energy dimension and send frames to FFmpeg
    for psi in psis:
        for phi in tqdm(phis, desc=f'Building MP4'):
            for theta in thetas:
                # Make & customize plot
                fig = plot_reciprocal3D_det_plotly(psi, phi, theta, iq, qx, qy, qz)

                buf = io.BytesIO()
                fig.write_image(buf, format='png', width=1000, height=1000, engine="kaleido")
                buf.seek(0)
                # Write the PNG buffer data to the process
                proc.stdin.write(buf.getvalue())
                plt.close('all')

    # Finish the subprocess
    out, err = proc.communicate()
    if proc.returncode != 0:
        print(f"Error: {err}")

In [ ]:
def dets_to_mp4(ordered_det_paths, det_h, det_v, output_path, frame_rate=15, quality=17):
    """
    Generate mp4 video of images along a specified dimension (e.g. energy, time). 
    Requires subprocess import. 
    
    Inputs:
    output_path (str or pathlib.Path): path to generated mp4 (includes mp4 filename)
    frame_rate (int, default=15): frame rate of mp4 generated
    quality (int, default=17): 'crf' quality value; lower is better, 17 is often considered visually lossless
    
    Outputs:
    mp4 movie file where specified in output path
    """

    # FFmpeg command. This is set up to accept data from the pipe and use it as input, with PNG format.
    # It will then output an H.264 encoded MP4 video.
    cmd = [
        'ffmpeg',
        '-y',  # Overwrite output file if it exists
        '-f', 'image2pipe',
        '-vcodec', 'png',
        '-r', str(frame_rate),  # Frame rate
        '-i', '-',  # The input comes from a pipe
        '-vcodec', 'libx264',
        '-pix_fmt', 'yuv420p',
        '-crf', str(quality),  # Set the quality (lower is better, 17 is often considered visually lossless)
        str(output_path)
    ]

    # Start the subprocess
    proc = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Loop through the energy dimension and send frames to FFmpeg
    for det_path in tqdm(ordered_det_paths, desc=f'Building MP4'):
        # Load the image into a variable
        img = np.load(det_path)

        #mine phi value
        psi = det_path.split('_psi')[1].split('_phi')[0]
        phi = det_path.split('_phi')[1].split('_theta')[0]
        theta = det_path.split('_theta')[1].split('.npy')[0]
        
        fig, ax1 = subplots()
        cax = ax1.imshow(img,
                   norm=matplotlib.colors.LogNorm(vmin=np.percentile(img, 50), vmax=np.percentile(img, 99.99)),
                   extent=(np.min(det_h),np.max(det_h),np.min(det_v),np.max(det_v)),
                   cmap='turbo',
                   origin = 'lower')
        ax1.set_title(f'Detector Psi={psi}°, Phi={phi}°, Theta={theta}°')
        ax1.set_xlabel('q horizontal $(Å^{-1})$')
        ax1.set_ylabel('q vertical $(Å^{-1})$')
        # ax1.set_xlim(left=0)
        # ax1.set_ylim(bottom=0)
        cbar = fig.colorbar(cax, ax=ax1, shrink=0.8, aspect=20, pad=0.02)
        
        buf = io.BytesIO()
        plt.savefig(buf, dpi=300, format='png')
        buf.seek(0)
        # Write the PNG buffer data to the process
        proc.stdin.write(buf.getvalue())
        plt.close('all')

    # Finish the subprocess
    out, err = proc.communicate()
    if proc.returncode != 0:
        print(f"Error: {err}")

In [ ]:
def plot_reciprocal3D_det_plotly(psi, phi, theta, iq, qx, qy, qz):
    det_pixels = (200,200) #horizontal, vertical
    det_qs = (6,4) #horizontal, vertical (these are absolute maximums. detector centered at 0)
    det_x_grid, det_y_grid, det_z_grid, det_h, det_v = make_detector(det_qs[0], det_pixels[0], det_qs[1], det_pixels[1])
    
    
    det_x_grid, det_y_grid, det_z_grid = rotate_about_normal(det_x_grid, det_y_grid, det_z_grid, psi)
    det_x_grid, det_y_grid, det_z_grid = rotate_about_vertical(det_x_grid, det_y_grid, det_z_grid, phi)
    det_x_grid, det_y_grid, det_z_grid = rotate_about_horizontal(det_x_grid, det_y_grid, det_z_grid, theta)
    
    threshold_pct = 99.7
    density_grid = iq
    x_axis = qx 
    y_axis = qy 
    z_axis = qz
    
    y, x, z = np.where(density_grid>np.percentile(density_grid, threshold_pct))
    values = density_grid[y, x, z]
    
    max_values = np.max(values)
    min_values = np.min(values)
    # Get the absolute coordinates
    x_abs = x_axis[x]
    y_abs = y_axis[y]
    z_abs = z_axis[z]


    fig = go.Figure()

    # Plot the detector surface
    fig.add_trace(go.Surface(x=det_x_grid, 
                             y=det_y_grid, 
                             z=det_z_grid, 
                             colorscale=[[0, 'aquamarine'], [1, 'aquamarine']], 
                             opacity=1, 
                             showscale=False))
    # Calculate the opacity for the current level
    opacity = 0.3
    color = 'fuchsia'
    
    # Scatter plot for the current subset of data
    fig.add_trace(go.Scatter3d(
        x=x_abs,
        y=y_abs,
        z=z_abs,
        mode='markers',
        showlegend=False,
        marker=dict(
            size=2,
            opacity=opacity,
            color=color)))

    fig.add_trace(go.Scatter3d(
            x=[None],
            y=[None],
            z=[None],
            mode="markers",
            name="Detector Plane",
            marker=dict(size=20, color="aquamarine", symbol='square')))

    fig.add_trace(go.Scatter3d(
            x=[None],
            y=[None],
            z=[None],
            mode="markers",
            name="Bragg Peaks",
            marker=dict(size=20, color="fuchsia", symbol='square')))

    camera = dict(
        eye=dict(x=1.5, y=1.5, z=1.5))
    
    fig.update_layout(showlegend=True)
    range_val = 7
    
    fig.update_layout(
        scene=dict(
            xaxis=dict(
                title='q<sub>x</sub> (Å<sup>-1</sup>)',
                titlefont=dict(size=20, color='black'),
                tickfont=dict(size=15, color='black'),
                backgroundcolor="white",
                color="black",
                gridcolor="black",
                range = (-range_val,range_val)
            ),
            yaxis=dict(
                title='q<sub>y</sub> (Å<sup>-1</sup>)',
                titlefont=dict(size=20, color='black'),
                tickfont=dict(size=15, color='black'),
                backgroundcolor="white",
                color="black",
                gridcolor="black",
                range = (-range_val,range_val)
            ),
            zaxis=dict(
                title='q<sub>z</sub> (Å<sup>-1</sup>)',
                titlefont=dict(size=20, color='black'),
                tickfont=dict(size=15, color='black'),
                backgroundcolor="white",
                color="black",
                gridcolor="black",
                range = (-range_val,range_val)
            ),
            # aspectmode='data',
            camera=camera  # Apply the camera settings
        ),
        paper_bgcolor="white",
        plot_bgcolor="white",
        legend=dict(
            x=0.65,  # Adjust x to move the legend horizontally
            y=0.86,  # Adjust y to move the legend vertically
            font=dict(size=20, color='black'),
            bgcolor="rgba(255, 255, 255, 0.7)")
        )

      # Define camera settings to zoom out


    return fig

In [ ]:
%%time
dirr = os.getcwd()
xyz_path = f'{dirr}/test_xyz_files/graphite_medium.xyz'
buffer = 0.5
voxel_size = 0.25
dens_grid, x_axis, y_axis, z_axis = generate_density_grid(xyz_path, voxel_size, min_ax_size=512)

In [ ]:
%%time
iq, qx, qy, qz = convert_grid_qspace(dens_grid, x_axis, y_axis, z_axis)

In [ ]:
# optional downselect iq meshgrid based on max q desired
max_q = 6
iq_small, qx_small, qy_small, qz_small = downselect_meshgrid(iq, qx, qy, qz, max_q)

#optional free up memory
del iq
del dens_grid

#reassign variables
iq = iq_small
qx = qx_small
qy = qy_small
qz = qz_small

#apply debye waller real-space gaussian smearing
sigma = 0.2
iq = multiply_ft_gaussian(iq, qx, qy, qz, sigma)

In [ ]:
%%time
dirr = os.getcwd()
save_path = f'{dirr}/det_output_files/'
if not os.path.exists(save_path):
    os.mkdir(save_path)
    
#setup detector
det_pixels = (300,200) #horizontal, vertical
det_qs = (6,4) #horizontal, vertical 
#(these are absolute maximums. detector centered at 0)
det_x, det_y, det_z, det_h, det_v = make_detector(det_qs[0], det_pixels[0], det_qs[1], det_pixels[1])
np.save(f'{save_path}det_h.npy', det_h)
np.save(f'{save_path}det_v.npy', det_v)

#initial detector rotation to align detector. 
#Normal axis of detector should be axis for tilting texture of real-space.
#Vertical axis of detector should be axis for fiber-like texture of real-space
psi_init = 0 #rotation in degrees of detector about detector normal axis
phi_init = 0 #rotation in degrees of detector about detector vertical axis
det_x, det_y, det_z = rotate_about_normal(det_x, det_y, det_z, psi_init)
det_x, det_y, det_z = rotate_about_vertical(det_x, det_y, det_z, phi_init)

#set up rotations to capture disorder in your film. psi=tilting, phi=fiber texture
#only need 1/4 of your total rotation space since symmetry allows us to mirror quadrants
psis = np.linspace(0,1,num=1) #rotation in degrees of detector about detector normal axis
phis = np.linspace(0,180,num=181) #rotation in degrees of detector about detector vertical axis
theta = 0 #rotation in degrees of detector about detector horizontal axis

args = [(iq, qx, qy, qz, det_h, det_v, det_x, det_y, det_z, psi, phi, theta, save_path) for psi in psis for phi in phis]
with Pool(processes=8) as pool:
    filenames = pool.map(generate_detector_ints, args)

In [ ]:
import subprocess
import io
import gc

In [ ]:
%%time
output_path = '/Users/Thomas2/Downloads/test45.png'
fig = plot_reciprocal3D_det_plotly(0, 45, 0, iq, qx, qy, qz)
fig.write_image(output_path, width=1000, height=1000, engine="kaleido")

In [ ]:
psis = np.linspace(0,1, num=1)
phis = np.linspace(0,359, num=360)
thetas = np.linspace(0,1, num=1)
output_path = '/Users/Thomas2/Downloads/rotation.mp4'
iq_to_mp4(psis, phis, thetas, iq, qx, qy, qz, output_path, frame_rate=15, quality=17)

In [ ]:
dirr = os.getcwd()
save_path = f'{dirr}/det_output_files/'
det_h = np.load(f'{save_path}det_h.npy')
det_v = np.load(f'{save_path}det_v.npy')
files = glob.glob(f'{save_path}det_ints*.npy')
ordered_det_paths = sorted(
    [f for f in files if f.startswith(f'{save_path}det_ints_psi') and f.endswith('.npy')],
    key=lambda x: int(x.split('_phi')[1].split('_theta')[0])
)
output_path = '/Users/Thomas2/Downloads/det_movie.mp4'
dets_to_mp4(ordered_det_paths, det_h, det_v, output_path, frame_rate=15, quality=5,)

In [ ]:
# psi = 0 #rotation in degrees of detector about detector normal axis
# phi = 0 #rotation in degrees of detector about detector vertical axis
# theta = 0 #rotation in degrees of detector about detector horizontal axis

det_pixels = (200,200) #horizontal, vertical
det_qs = (6,4) #horizontal, vertical (these are absolute maximums. detector centered at 0)
det_x_grid, det_y_grid, det_z_grid, det_h, det_v = make_detector(det_qs[0], det_pixels[0], det_qs[1], det_pixels[1])


det_x_grid, det_y_grid, det_z_grid = rotate_about_normal(det_x_grid, det_y_grid, det_z_grid, psi)
det_x_grid, det_y_grid, det_z_grid = rotate_about_vertical(det_x_grid, det_y_grid, det_z_grid, phi)
det_x_grid, det_y_grid, det_z_grid = rotate_about_horizontal(det_x_grid, det_y_grid, det_z_grid, theta)

threshold_pct = 99.7
num_levels=20
cmap = 'plasma'
density_grid = iq
x_axis = qx 
y_axis = qy 
z_axis = qz

y, x, z = np.where(density_grid>np.percentile(density_grid, threshold_pct))
values = density_grid[y, x, z]

max_values = np.max(values)
min_values = np.min(values)
# Get the absolute coordinates
x_abs = x_axis[x]
y_abs = y_axis[y]
z_abs = z_axis[z]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Define the number of levels of opacity
opacities = np.linspace(0.3,0.01,num_levels)

cmap = plt.get_cmap(cmap)
colors = cmap(np.linspace(0.3, 1, num_levels))

for i in range(num_levels):
    # Calculate the opacity for the current level
    opacity = opacities[i]
    color = colors[i]

    mask_low = 100*i/num_levels
    mask_high = 100*(i+1)/num_levels
    # Determine the data points that fall into the current opacity level
    mask = (values > np.percentile(values, mask_low)) & (values <= np.percentile(values, mask_high))
    
    # Scatter plot for the current subset of data
    ax.scatter(x_abs[mask], 
               y_abs[mask], 
               z_abs[mask], 
               color=color,  # Use the single color for all points
               alpha=opacity, 
               edgecolor='none')

# Set labels and titles
ax.set_xlabel('qx')
ax.set_ylabel('qy')
ax.set_zlabel('qz')
# ax.set_title('3D Scatter Plot of Electron Density')

# Setting equal aspect ratio
max_range = np.array([x_abs.max()-x_abs.min(), y_abs.max()-y_abs.min(), z_abs.max()-z_abs.min()]).max() / 2.0
mid_x = (x_abs.max()+x_abs.min()) * 0.5
mid_y = (y_abs.max()+y_abs.min()) * 0.5
mid_z = (z_abs.max()+z_abs.min()) * 0.5
ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)

fig.patch.set_facecolor('black')  # Set the outer background color
ax.set_facecolor('black')  # Set the background color of the plot
# Change the color of the ticks and labels to white
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')
ax.zaxis.label.set_color('white')
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')
ax.tick_params(axis='z', colors='white')

# Change grid and pane colors
ax.xaxis.pane.set_edgecolor('white')
ax.yaxis.pane.set_edgecolor('white')
ax.zaxis.pane.set_edgecolor('white')
ax.grid(color='white', linestyle='--', linewidth=0.5)
ax.plot_surface(det_x_grid, det_y_grid, det_z_grid, alpha=0.5, color='green')